In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
DATA_PATH = os.getenv("DATA_PATH")

In [2]:
import pandas as pd

df = pd.read_csv(DATA_PATH + "articles_testing.tsv", sep="\t")
df

,content
0,"Vào tối 22/10, sau đêm bán kết của cuộc thi Ho..."
1,"Tân Hoa Xã đưa tin rằng vào ngày 9/10, phát ng..."
2,Chuyến thăm của Lloyd Austin diễn ra chỉ vài n...
3,"Trong dự thảo Luật Nhà giáo, Bộ GD&ĐT đã đưa r..."
4,"Trong buổi gặp gỡ, Chủ tịch Hà Thị Nga đã bày ..."
...,...
43898,"Giá vàng tiếp tục tăng phiên thứ hai, đạt đỉnh..."
43899,Để tỏ lòng tri ân với lịch sử và những người c...
43900,MangoTV vừa công bố danh sách phim có lượt xem...
43901,Để hoạt động đấu giá khoáng sản minh bạch và h...


In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import torch

model = AutoModelForTokenClassification.from_pretrained(
    "./xlm-roberta-keywordtagger")
tokenizer = AutoTokenizer.from_pretrained("./xlm-roberta-keywordtagger")
# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model.to(device)

nlp = pipeline("ner", model=model, tokenizer=tokenizer, device='cuda')

example = df['content'][0]
# Print len token
print(len(tokenizer.tokenize(example)))


def get_ner_results(text):
    ner_results = nlp(text)
    keywords = []
    current_word = ""
    for result in ner_results:
        if result['entity'] == 'B':
            if current_word != "":
                keywords.append(current_word)
                current_word = ""
            current_word = result['word'].strip('▁')
        if result['entity'] == 'I':
            if result['word'][0] == '▁':
                current_word += " " + result['word'].strip('▁')
            else:
                current_word += result['word']

    # Add the last word
    if current_word != "":
        keywords.append(current_word)

    return set(keywords)


def process_long_text(text: str, tokenizer, max_length=512):
    # Check if text needs splitting
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    input_ids = tokens['input_ids'][0]

    if len(input_ids) <= max_length:
        return ','.join(get_ner_results(text))

    # Split into sentences
    sentences = text.split('.')
    sentences = [s.strip() + '.' for s in sentences if s.strip()]

    # Combine sentences into chunks
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        # Check length with new sentence
        test_text = ' '.join(current_chunk + [sentence])
        test_tokens = tokenizer(test_text, return_tensors="pt")
        test_length = len(test_tokens['input_ids'][0])
        if test_length > max_length:
            continue
        if current_length + test_length <= max_length:
            current_chunk.append(sentence)
            current_length += test_length
        else:
            # Save current chunk and start new one
            if current_chunk:
                chunks.append(' '.join(current_chunk))
            current_chunk = [sentence]
            current_length = len(
                tokenizer(sentence, return_tensors="pt")['input_ids'][0])

    # Add final chunk
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    # Process chunks and combine results
    all_entities = set()
    for chunk in chunks:
        chunk_entities = get_ner_results(chunk)
        all_entities.update(chunk_entities)

    return ','.join(all_entities)

363


In [ ]:
# Process all articles
df['tags'] = df['content'].apply(
    lambda x: process_long_text(x, tokenizer))

df.to_csv('./nhom1_sol2.tsv', sep='\t', index=False)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
